# Quickstart tutorial: Enter the _Vivarium_

First follow the installations instruction in the README of the repository. If everything works correctly, you should have launched the web interface.

## Introduction to the web interface

For the web interface tutorial, please refer to [interface.md](interface.md).

## Controlling the simulator programmatically

Instead of controlling the simulator from the web interface, you can also control it programmatically, e.g. directly from this Jupyter notebook. First make sure that the simulator is stopped in the web interface. Then execute:

In [1]:
from vivarium.controllers.notebook_controller import NotebookController
import numpy as np
controller = NotebookController()

INFO:vivarium.simulator.behaviors:{'FEAR': 0, 'AGGRESSION': 1, 'LOVE': 2, 'SHY': 3, 'manual': 4, 'noop': 5}


The newly created `controller` object provides all the functionalities to easily interact with the simulator server. Let's look for example at the agents present in the environment:

In [ ]:
controller.agents

This is a list of `Agent` objects, one per agent in the environment. Let's for example access the configuration of the first agent in this list and look at its attribute using the Python `.__dict__` syntax:

In [ ]:
controller.agents[0].__dict__

We see the value of different attributes of the agent, for example its position, diameter or color. Let's make the first agent larger by increasing its `diameter` attribute:

In [ ]:
controller.agents[0].diameter = 10.

If you now look at the web interface, you should see that one of the agent is now twice larger than the others. All changes made in this notebook will be automatically visible in the web interface, and vice versa. One can for example also change the color of an agent, let's do it on the third agent in the list now (i.e. the agent with index 2):

In [11]:
controller.agents[2].color = 'red'

INFO:vivarium.controllers.simulator_controller:push_state 1


The same principles apply to objects, which are accessible through `controller.objects`.

### Controlling the agent's movement

We can modify the appearance of the agents, but we can also decide how they move. For now, let's set the speed of the right wheel of the first agent. First, we need to start the server with this instruction:

In [ ]:
controller.run(threaded=True)

INFO:vivarium.controllers.simulator_controller:push_state 0


INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_controller:push_state 0
INFO:vivarium.controllers.simulator_cont

We can stop it at any time by executing the following:

In [9]:
controller.stop()

If you stopped the server, restart it now then execute the following code to set the right wheel of the first agent to 1 radian per second:

In [ ]:
controller.agents[0].left_motor = 1.

You should now see the agent spinning. To make it move straight, we need to set the other wheel to the same speed, with the following:

In [ ]:
controller.agents[0].right_motor= 1.

Once done, we can stop the server.

In [ ]:
controller.stop()

### Retrieving the proximeters values

The agents also come with a set of sensors called `proximeters`, that activate when the agent approaches another entity.

We can retrieve the value of these sensors with this instruction:

In [15]:
print(f"right prox: {controller.agents[0].right_prox}")
print(f"left prox: {controller.agents[0].left_prox}")

right prox: 0.9067567586898804
left prox: 0.7911147475242615
